### 1. Checklist

###### Checklist for logs: 

|job ID|log| reason acc. to elasticsearch | reason acc to script| |
|-|----------|----------|----------|-----|
| baf77c21-bdd5-0aa2-8731-000000000385|9ef7ce17ae108085e8a26e1f2046e1cf73e8c976.json|Infrastructure - Matlab License |Infrastructure - Matlab License |y|
| 86623622-6e74-9368-74a8-000000000028|882dafcc748b4f0695ce486241a05ad2.json|?| Test(s) failed|y|
|16911dd9-d58b-df18-7329-000000000ba3|e850514f59cb3334754f2641fc63f20f46af56e7.json|FuSa Violation found|Main Category: FuSa Violation Found (Check 'bazel_wrapper_log.txt' on the logs folder), Subcategory: , Pattern: Failed actions detected in bazel_wrapper_log.txt file|y|
|No ID provided||?|redirected output - Fusa Small Checks - Check-Bazel-rules-in-FuSa-critical-components, Subcategory: Fusa Small Checks - Check-Bazel-rules-in-FuSa-critical-components|?|
|No ID provided||?| redirected output - default, Subcategory: default|?|
|c65a4cd1-edf7-790d-cd51-000000000042|0752ac127d21a70af1765ae29afc53d978224f25.json|python|Test(s) failed, Subcategory: |?|
|No ID provided||python|Main Category: python, Subcategory: ,|y||
|b6d2a4bc-700f-7546-f838-000000000042|8e89be688f5b331654a6f44975a7c6c23dd3b71c.json|Compiler (gnu)  | Bazel - error executing command, Subcategory: error executing command|X|
||00e0b3d030f7fd0eef0c4b016e3d5e7e2b51a69f.json|Bazel (failed on target analysis)|Not flagged / No match found|X|
|No ID provided|b819bae8dfb1923b8f9dc3c1b4bb2eed8c3fa2a3.json|Compiler (clang)|Bazel - error executing command, Subcategory: error executing command|X|
|NO ID provided|cd5c0bbd5821d9d31f41442f78bcae86221e65a0.json|Bazel(missing input file)|Bazel - missing input file, Subcategory: missing input file, Pattern: ERROR:(.*)input file\(s\) do not exist |?|
|NO ID provided||Bazel(missing input file)|Bazel - missing input file, Subcategory: missing input file, Pattern: ERROR:(.*)missing input file '.*' |?|

Why same file gets "unknown error" & a match? -->  Because 18 logs but 34 errors - Needs Dealing with
- 005fc583d4745e2470010c57b6422bcbc5c5fb0e271a7f11d72248e702e1090a (2 times)
- 0067a8788a1a5e30cecc3c159a8f5b6546a46cd5fb18253c82de30e8d1e7a988 (4 times)
- 0077401c9568efb6267a2c6ffd1e696ffe70e479d7c5e69413d3d7aea3105035 (4 times)
- e850514f59cb3334754f2641fc63f20f46af56e7 (6 times)
- 0079e5bcee8a7318e88e6d40be15c242ad522e2af6cbc1cdbeecf4beb0e743c7 (3 times)
- 0752ac127d21a70af1765ae29afc53d978224f25 (4 times)
- 8e89be688f5b331654a6f44975a7c6c23dd3b71c (3 times)
- 9ef7ce17ae108085e8a26e1f2046e1cf73e8c976 (2 times)
- cd5c0bbd5821d9d31f41442f78bcae86221e65a0 (2 times)

- fix inconsistencies in pattern? 
- fix script for all cases ( decision!) --> use existing _find_errors ()

### 2. code snippets i deemed important

In [ ]:
# checking preprocessed_logs against pattern
import json
import error_patterns
import re
import os
import textwrap

INFRA_PATTERNS = error_patterns.INFRA_PATTERNS
BUILD_PATTERNS = error_patterns.BUILD_PATTERNS

# compile patterns from error_patterns.py
def compile_patterns(patterns_dict):
    compiled_patterns = {}
    for main_category, patterns in patterns_dict.items():
        if isinstance(patterns, dict):
            for sub_category, sub_patterns in patterns.items():
                for pattern in sub_patterns:
                    compiled_patterns.setdefault((main_category, sub_category), []).append(re.compile(pattern))
        else:
            for pattern in patterns:
                compiled_patterns.setdefault((main_category, ""), []).append(re.compile(pattern))
    return compiled_patterns


# check log entry against patterns
def check_log_entry(log_entry, compiled_patterns):
    matches = []
    for (main_category, sub_category), regex_list in compiled_patterns.items():
        for regex in regex_list:
            if regex.search(log_entry):
                category = f"{main_category} - {sub_category}" if sub_category else main_category
                matches.append((category, sub_category, regex.pattern))
    return matches

# process files
def process_log_files(directory_path, compiled_patterns):
    summary = {
        'tasks_summary': {},
        'msg_summary': {}
    }
    for filename in filter(lambda f: f.endswith('.json'), os.listdir(directory_path)):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            log_entries = json.load(file)
            for log in log_entries:
                task_id = log.get('id', 'No ID provided')
                name = log.get('name', 'No task name provided')
                msg = log['msg']
                task_key = f"{name} (ID: {task_id})"
                summary['tasks_summary'].setdefault(task_key, 0)
                summary['tasks_summary'][task_key] += 1
                summary['msg_summary'].setdefault(msg, 0)
                summary['msg_summary'][msg] += 1

                stdout_text = "\n".join(log.get('stdout_lines', []))
                matches = check_log_entry(stdout_text, compiled_patterns)
                for match in matches:
                    main_category, sub_category, pattern = match
                    print(f"Current File: {file_path}")
                    print(task_key)
                    print(f"Main Category: {main_category}, Subcategory: {sub_category}, Pattern: {pattern}")
    return summary

# format summary for readability
def format_summary_to_screen_width(summary, terminal_width=150):
    formatted_summary = ""
    for key, value in summary.items():
        if isinstance(value, dict):
            formatted_summary += f"{key}:\n"
            for sub_key, sub_value in value.items():
                wrapped_sub_value = textwrap.fill(str(sub_value), terminal_width - 4)
                formatted_summary += f"  {sub_key}: {wrapped_sub_value}\n"
        else:
            wrapped_value = textwrap.fill(str(value), terminal_width)
            formatted_summary += f"{key}: {wrapped_value}\n"
        formatted_summary += "-" * terminal_width + "\n"
    return formatted_summary

# Usage:
compiled_infra_patterns = compile_patterns(INFRA_PATTERNS)
compiled_build_patterns = compile_patterns(BUILD_PATTERNS)
directory_path = 'preprocessed_logs'
all_compiled_patterns = {**compiled_infra_patterns, **compiled_build_patterns}
logs_directory_path = 'logs'
output_directory_path = 'preprocessed_logs'

summary = process_log_files(directory_path, all_compiled_patterns)
formatted_summary = format_summary_to_screen_width(summary)
print(formatted_summary)

In [ ]:
# check against pattern for single file
import json

# Function to check a log entry against compiled patterns
def check_log_entry(log_entry, compiled_patterns):
    matches = []
    for (main_category, sub_category), regex_list in compiled_patterns.items():
        for regex in regex_list:
            if regex.search(log_entry):
                category = f"{main_category} - {sub_category}" if sub_category else main_category
                matches.append((category, sub_category, regex.pattern))
    return matches

# Function to process a log file
def process_log_file(file_path, compiled_patterns):
    with open(file_path, 'r', encoding='utf-8') as file:
        log_entries = json.load(file)
        for entry in log_entries:
            stdout_text = "\n".join(entry.get('stdout_lines', []))
            matches = check_log_entry(stdout_text, compiled_patterns)
            if matches:
                for main_category, sub_category, pattern in matches:
                    print(f"Main Category: {main_category}, Subcategory: {sub_category}, Pattern: {pattern}")
            else:
                print("No matches found")

#USAGE:
path_example_logfile = 'preprocessed_logs/e850514f59cb3334754f2641fc63f20f46af56e7_cropped.json'

# Assuming compiled_infra_patterns and compiled_build_patterns are already defined
all_compiled_patterns = {**compiled_infra_patterns, **compiled_build_patterns}
process_log_file(path_example_logfile, all_compiled_patterns)

In [ ]:
## **What is the difference(in error_patterns.py) b/w those that work / don't work? **

# - Python (correct): 
BUILD_PATTERNS = {
    "python": {
        "Authentication error": [r"^.*Traceback.+\n( .+\n)+.*pip/download.py.+handle_401.*$"],
        "guestfs_launch failed": [r"RuntimeError: guestfs_launch failed"],
        "HTTPError": [r"^requests.exceptions.HTTPError: .+$"],
        "pip": [r"ERROR: Pip failed:.*"],
        "pip install failed": [
            r"^ERROR: No matching distribution found for.*$",
            r"^Could not install packages due to an.*$",
        ],
        "Failed to install python dependencies": [
            r"^ERROR: could not install deps .+$",
            r"Preparing metadata \(setup\.py\): finished with status 'error'",
        ],
        "libguestfs": [r"RuntimeError: guestfs_launch failed.+$"],
        "pytest": [
            r"(?s)=========================== short test summary info ============================.*?\sin\s.*?=+"
        ],
        "thread": [r"^.*python.+\n( .+\n).*error: can't start new thread"],
        "": [
            r"^(.*Traceback.+\n( .+\n)+(?P<type>\S.+Error.*?):.+\n?(?P<details>(\s.+\n)*)?)+",
            r"ERROR: .*: This target is being built for Python 3 but \(transitively\) includes Python 2-only sources.",
            r"\/usr\/bin\/python.*: No module named .*$",
        ],
        "unrecognized arguments": [r"(?P<file>[\w-]+\.py): error: unrecognized arguments:.+$"],
        "Exception raised": [
            r"Exception:(.*)Check missing paths here(.*)",
        ],
    }
}

# - Compiler (gnu / clang ...) (not correct):
BUILD_PATTERNS = {
    "Compiler": {
        "clang-tidy": [
            r"".join(
                [
                    r"(?s)^ERROR: .*Running clang-tidy on file (?P<file>.*) failed: ",
                    r"\(Exit.*(?P<details>INFO: \d+ counting clang-tidy .*\nINFO:.*\n.*?",
                    r"At least one clang-tidy finding was treated as error)",
                ]
            )
        ],
        "clang": [r"^ERROR: .*: Compiling .* failed: \(Exit \d\): clang\+\+ failed: error executing command"],
        "gnu": [
            r"^ERROR: .*: Compiling .* failed: \(Exit \d\): x86_64-linux-gnu-g\+\+ failed: error executing command.*$"
        ],
    }
}

# - FuSa (correct but where does additional info stem from?):
BUILD_PATTERNS = {
    "FuSa Violation Found (Check 'bazel_wrapper_log.txt' on the logs folder)": [
        r"Failed actions detected in bazel_wrapper_log.txt file"
    ],
    "FuSa Violation Found (Check 'mitguard_report.html' on the logs folder)": [r"Mitguard check failure"],
    "FuSa Violation Found (Check the log on 'Nightwatch' folder)": [r"Nightwatch verdict: rejected\."]
}

In [ ]:
# NOTE: compiled_infra_pattern: 

{('Infrastructure', ''): [re.compile('Unexpected failure during module execution.'), re.compile('Failed to update apt cache'), re.compile('^.+extended fault data: .+'), re.compile('WebException .+'), re.compile('HTTP error \\d+ .+')], ('Infrastructure', 'Host unreachable'): [re.compile('(Data could not be sent to remote host|Make sure this host can be reached|No address associated with hostname).+$')], ('Infrastructure - Ansible', ''): [re.compile('Timeout \\(.*\\) waiting for privilege escalation prompt')], ('Infrastructure - Artifactory', ''): [re.compile('\\[Error\\].* Artifactory response: (?P<type>.+)\\n<.+>\\n(?P<details>[^<]+)'), re.compile('\\[Error\\].* Artifactory response: (?P<type>.+)(\\n(.+\\n)*.+errors[^{]+{\\n(?P<details>[^{}]+)})?'), re.compile('^.*http.client.IncompleteRead: IncompleteRead.*'), re.compile('unauthorized: The client does not have permission to push to the repository'), re.compile('ConnectionError: .+artifactory\\.cc\\.bmwgroup\\.net.*$'), re.compile('Failure downloading https:\\/\\/(.*)artifactory.cc.bmwgroup.net\\/(.*), Connection failure: The read operation timed out'), re.compile("Error downloading 'https:\\/\\/(.*)\\.artifactory\\.cc\\.bmwgroup\\.net(.*)The remote name could not be resolved: '(.*)\\.artifactory\\.cc\\.bmwgroup\\.net'"), re.compile('artifactory-cc-bmwgroup-net-artifactory.*connection reset by peer$')], ('Infrastructure - Artifactory', 'Bazel download error'): [re.compile('could not download Bazel:.+unable to complete request.+$')], ('Infrastructure - Artifactory', 'Download error'): [re.compile('could not download .* failed with error.*$')], ('Infrastructure - Artifactory', 'Unexpected EOF'): [re.compile('could not download .* unexpected EOF$')], ('Infrastructure - Artifactory', 'Checksum mismatch'): [re.compile('Error in download: java\\.io\\.IOException: Error downloading \\[.+\\] to .+: Checksum was [a-f0-9]+ but wanted [a-f0-9]+')], ('Infrastructure - Artifactory', 'Content-Length mismatch'): [re.compile('Error downloading\\s\\[http(s)?:\\/\\/(\\w+)\\.artifactory\\.cc\\.bmwgroup\\.net\\/artifactory\\/.* Bytes read [0-9]* but wanted [0-9]*')], ('Infrastructure - Artifactory', 'Operation failed'): [re.compile('^.*Non-zero return code from art_cli:.*')], ('Infrastructure - Artifactory', 'Operation timed out'): [re.compile("Error downloading 'https:\\/\\/(.*)\\.artifactory\\.cc\\.bmwgroup\\.net(.*)to(.*)\\: The operation has timed out")], ('Infrastructure - Artifactory', 'Permission'): [re.compile('403 Client Error: Forbidden for url.*')], ('Infrastructure - Artifactory', 'Unknown host (ddad)'): [re.compile('Unknown\\shost:\\sddad\\.artifactory\\.cc\\.bmwgroup\\.net')], ('Infrastructure - Artifactory', 'Unknown host (codecraft)'): [re.compile('Unknown\\shost:\\scodecraft(.*)artifactory(.*)\\.cc\\.bmwgroup\\.net')], ('Infrastructure - Artifactory', 'Fetch failed'): [re.compile('ERROR:.+An error occurred during the fetch of repository.+(\\n.+){1,5}\\nError in download.+'), re.compile('^.*E: Failed to fetch https:\\/\\/.*artifactory\\.cc\\.bmwgroup\\.net\\/.* File has unexpected size \\(.* != .*\\)\\. Mirror sync in progress\\?.*$')], ('Infrastructure - Artifactory', 'Service unreachable'): [re.compile('artifactory\\.cc\\.bmwgroup\\.net[^\\s]*\\s(?P<details>[^\\(]*\\(?Service Unavailable\\)?)')], ('Infrastructure - Bazel remote cache', 'Inaccessible remote cache'): [re.compile('WARNING: Writing to Remote Cache:\\nio.grpc.StatusRuntimeException: UNKNOWN: operation not permitted'), re.compile('WARNING: Writing to Remote Cache:\\nio.grpc.StatusRuntimeException: UNKNOWN: EOF'), re.compile('ERROR: Failed to query remote execution capabilities: UNAVAILABLE: Unable to resolve host buildbarn\\.cc\\.bmwgroup\\.net'), re.compile('^java.io.IOException: io.grpc.StatusRuntimeException: UNAVAILABLE: Unable to resolve host buildbarn.cc.bmwgroup.net$')], ('Infrastructure - Bazel remote cache', 'Build without the bytes'): [re.compile("^ERROR: .* Failed to fetch file with hash '.*' because it does not exist remotely\\..*$"), re.compile("^(?:\\s+|\\\\t)?(Suppressed: )?java\\.io\\.IOException: Failed to fetch file with hash '.*' because it does not exist remotely\\..*$"), re.compile('ERROR: .* failed: failed to create _solib symbolic link .* to target .*: Missing digest: .*'), re.compile('ERROR: .*: Exec failed due to IOException: null'), re.compile('^ERROR: .*io\\.grpc\\.StatusRuntimeException: UNAVAILABLE: io exception.*$'), re.compile('^(java.io.IOException:|ERROR:).*io.grpc.StatusRuntimeException: UNAVAILABLE: upstream connect error or disconnect/reset before headers.*$')], ('Infrastructure - Bazel remote cache', 'Connection closed'): [re.compile('^.*Suppressed: java.io.IOException: An existing connection was forcibly closed by the remote host.*$')], ('Infrastructure - Bazel remote cache', 'Connection timed out'): [re.compile('ERROR: Failed to query remote execution capabilities: [Cc]onnection timed out.+$')], ('Infrastructure - Bazel remote cache', 'Connection refused'): [re.compile('ERROR: Failed to query remote execution capabilities: Connection refused'), re.compile('ERROR: Failed to query remote execution capabilities: finishConnect\\(\\.\\.\\) failed: Connection refused: buildbarn\\.cc\\.bmwgroup\\.net/.*$'), re.compile('^Caused by:.*UNAVAILABLE:.*connection error'), re.compile('^.*UNAVAILABLE.*connection error.*connection refused.*')], ('Infrastructure - Bazel remote cache', 'Deadline exceeded'): [re.compile('^.*DEADLINE_EXCEEDED:.*deadline exceeded.*')], ('Infrastructure - Bazel remote cache', 'Handshake timed out'): [re.compile('ERROR: Failed to query remote execution capabilities: handshake timed out after.+$')], ('Infrastructure - Bazel remote cache', 'Invalid Certificate'): [re.compile("^ERROR: Failed to init TLS infrastructure using '.*' as client certificate: File does not contain valid certificates: .*")], ('Infrastructure - Bazel remote cache', 'No route to host'): [re.compile('ERROR: Failed to query remote execution capabilities: finishConnect\\(\\.\\.\\) failed: No route to host:.+$')], ('Infrastructure - Bazel remote cache', 'No healthy upstream'): [re.compile('^java.io.IOException: io.grpc.StatusRuntimeException: UNAVAILABLE: no healthy upstream$'), re.compile('^ERROR: .*io\\.grpc\\.StatusRuntimeException: UNAVAILABLE: no healthy upstream$')], ('Infrastructure - Bazel remote cache', 'Resource exhausted'): [re.compile('^ERROR:.*io\\.grpc\\.StatusRuntimeException: RESOURCE_EXHAUSTED.*$')], ('Infrastructure - Bazel remote cache', 'Resource not found'): [re.compile('^.*Traceback.+\\n( .+\\n)+.+ResourceNotFound.*')], ('Infrastructure - Bazel remote cache', 'Generic status runtime exception'): [re.compile('^ERROR:.+io\\.grpc\\.StatusRuntimeException:.+$')], ('Infrastructure - Bazel Remote Execution', 'Operation not found'): [re.compile('^ERROR: .+\\(Exit 34\\): NOT_FOUND: Operation not found:.+$')], ('Infrastructure - Bazel Remote Execution', 'Docker pull failed'): [re.compile('^ERROR: .+\\(Exit 34\\): INVALID_ARGUMENT: Unable to start docker container.+$')], ('Infrastructure - BES upload', 'Failure'): [re.compile('^ERROR: The Build Event Protocol upload failed.*')], ('Infrastructure - BES upload', 'Timeout'): [re.compile('^ERROR: The Build Event Protocol upload timed out.*')], ('Infrastructure - Bazel test logs reporter', ''): [re.compile('bazel_test_logs_reporter - Unexpected error: (?P<details>.+)')], ('Infrastructure - Git LFS', ''): [re.compile('^.*Request Time-out.*'), re.compile('batch response: Fatal error: (?P<details>.+)')], ('Infrastructure - PyPI server not responding (possible AF auth failure)', ''): [re.compile('Could not find a version that satisfies the requirement .+')], ('Infrastructure - Git', ''): [re.compile('Unable to checkout .+ in submodule path .+'), re.compile('error: failed to push .+'), re.compile("ERROR:.+An error occurred during the fetch of repository.+(\\n.+){1,}\\nError in fail: error running 'git fetch.+"), re.compile('fatal:.+Could not read from remote repository.+')], ('Infrastructure - Docker', ''): [re.compile('=+\\n.+\\n=+\\n+Cannot connect to the Docker daemon .+'), re.compile('=+\\n.+\\n=+\\n+Error response .+'), re.compile('^.*Error: unable to pull ddad.artifactory.cc.bmwgroup.net.*'), re.compile('Error\\: error logging into \\"ddad\\.artifactory\\.cc\\.bmwgroup\\.net\\"\\: invalid username\\/password'), re.compile('Error response from daemon(.*)artifactory\\.cc\\.bmwgroup\\.net[^\\s]*\\s(?P<details>.*)'), re.compile('manifest for artifactory\\.cc\\.bmwgroup\\.net\\/.*\\:latest not found\\: manifest unknown\\: The named manifest is not known to the registry'), re.compile('^Error: Error parsing image configuration: unable to retrieve auth token: invalid username/password: unknown: Bad props auth token(:?).*$')], ('Infrastructure - Tasking licenses exhausted', ''): [re.compile('^(ctc|ltc) .*E109'), re.compile('^ERROR: (ctc|ltc) .*: protection error: .*License expired\\. No valid license found for.*')], ('Infrastructure - ZipArchive', ''): [re.compile('^failed to unpack .* to .*$')], ('Infrastructure - OpenShift CLI', ''): [re.compile('^Error from server \\(.*\\): .+$')], ('Infrastructure - Winrm failure', ''): [re.compile('winrm connection error: .+'), re.compile('winrm send_input failed'), re.compile('winrm connection error: HTTPSConnectionPool')], ('Infrastructure - xpad-build-integration', 'Copy ITF test images to cache server'): [re.compile("^cp: failed to close '\\/mnt\\/itf-images\\/.*': Input\\/output error$")], ('Infrastructure - xpad-build-integration', 'Signing server unresolvable'): [re.compile('ssh: Could not resolve hostname xpad-ci-dev.sign.extern.cc.bmwgroup.net: Temporary failure in name resolution')], ('Infrastructure - xpad-build-integration', 'Signing server connection error'): [re.compile('connect to host xpad-ci-dev.sign.extern.cc.bmwgroup.net port 22: No route to host')], ('Infrastructure - xpad-build-integration', 'Signing failure'): [re.compile('^subprocess\\.CalledProcessError: Command .*/.xpad_sign\\/bin\\/sign.* returned non-zero exit status [1-9]*\\.$'), re.compile('^.*signature generation failed$')], ('Infrastructure - License Server', ''): [re.compile('FlexNet Licensing error:.*\\n.*'), re.compile('ERROR:(.*)Not able to obtain FlexLM license(.*)')], ('Infrastructure - License Server (CAS)', ''): [re.compile('ERROR:(.*)License request for vsxrtegenerator feature failed(.*)')], ('Infrastructure - License Server (QNX)', ''): [re.compile('Feature:.*kwbuildproject.*\\n.*License path:.*([0-9]{4})@([a-z+0-9]+).muc:.*\\n.*')], ('Infrastructure - Network', 'License'): [re.compile('^.*: protection error: .*Hostname lookup failed .* No valid license found for.*')], ('Infrastructure - Network', 'Traffic Monitoring'): [re.compile('^Could not find the requested service network_traffic_monitoring\\: host.*')], ('Infrastructure - IOException', ''): [re.compile('^.*Exec failed due to IOException.*')], ('Infrastructure - netrc', ''): [re.compile('ERROR: There was a problem ensuring netrc entries')], ('Infrastructure - Matlab License', ''): [re.compile('License checkout failed.')]}


In [ ]:
# NOTE compiled_build_pattern

{('test-msg', ''): [re.compile('Check if msg is being posted to PR')], ('cc-test', ''): [re.compile('CC-ERROR: .*')], ('tresos', ''): [re.compile('Error \\d+: ERROR .+? \\((?P<type>.+?)\\) .+')], ('gtest', ''): [re.compile('^(?P<file>.+?):(?P<line>\\d+): Failure.*\\n(?P<details>[^.[]*)(\\[  FAILED  \\].+)?'), re.compile('^(?P<file>.+?)\\((?P<line>\\d+)\\): error: (?P<details>.+\\n([^.[].+\\n)*)(\\[  FAILED  \\].+)?')], ('CTest', ''): [re.compile('Errors while running CTest')], ('Codeowners Validator', 'Duplicated Pattern Checker'): [re.compile('\\s+\\[err\\]\\s+Pattern .* is defined \\d+ times in lines:.*')], ('Codeowners Validator', 'File Exist Checker'): [re.compile('\\s+\\[err\\]\\sline\\s\\d+:.*does not match any files in repository')], ('Codeowners Validator', 'Valid Syntax Checker'): [re.compile('CODEOWNERS group .*does not match regex expression.*and not found in.*file')], ('Codeowners Validator', 'Not Owned File Checker'): [re.compile('\\s+\\[err\\](.*)Found (\\d*) not owned files')], ('Codify', 'MappingsCheck - Violation detected'): [re.compile('ERROR Mappings.*Check - Violation detected')], ('Codify', 'Code generation'): [re.compile('ERROR(.*)Codify generating code(.*)failed(.*)codify.exe failed(.*)'), re.compile('ERROR(.*)Generating resolved json(.*)failed(.*)codify.exe failed(.*)'), re.compile('ERROR(.*)Generating codify-data for(.*)failed(.*)codify_bin failed(.*)')], ('Codify', 'Unresolved references'): [re.compile('ERROR DefaultLogger \\- All references have to be resolved! .*? are unresolved\\:')], ('Coverage', ''): [re.compile('fatal error: test coverage is too low for target .*')], ('Compiler', 'clang-tidy'): [re.compile('(?s)^ERROR: .*Running clang-tidy on file (?P<file>.*) failed: \\(Exit.*(?P<details>INFO: \\d+ counting clang-tidy .*\\nINFO:.*\\n.*?At least one clang-tidy finding was treated as error)', re.DOTALL)], ('Compiler', 'clang'): [re.compile('^ERROR: .*: Compiling .* failed: \\(Exit \\d\\): clang\\+\\+ failed: error executing command')], ('Compiler', 'gnu'): [re.compile('^ERROR: .*: Compiling .* failed: \\(Exit \\d\\): x86_64-linux-gnu-g\\+\\+ failed: error executing command.*$')], ('Compiler', 'gcc - Failure while compiling C++'): [re.compile('^(?P<file>.+?):(?P<line>\\d+).*: ERROR:.+BUILD(.bazel)?:[0-9]+:[0-9]+: C\\+\\+.*gcc failed.*$'), re.compile('^(?P<file>.+?):(?P<line>\\d+).*: ERROR:.+BUILD(.bazel)?:[0-9]+:[0-9]+: C\\+\\+ compilation of rule.* failed.*$'), re.compile('^(?P<file>.+?):(?P<line>\\d+).*: C\\+\\+ compilation of rule.* failed.*$')], ('Compiler', 'qcc - Failure executing command'): [re.compile('^ERROR:.+ Compiling application.+qcc failed: error executing command.+\\)')], ('Compiler', 'ghs'): [re.compile('^(\\S.+\\n)?"(?P<file>.+?)", line (?P<line>\\d+): (?P<type>(fatal )?error #.+)\\n(?P<details>(\\s.+\\n)+)'), re.compile('FAILED: (?P<file>.+)\\n.+\\n(?P<details>FATAL ERROR: .+)')], ('Compiler', 'NDS'): [re.compile('^ERROR.*Converting physical model from NDS to C\\+\\+ Headers')], ('Compiler', 'ltc'): [re.compile('^.*ltc E106: unresolved external.*')], ('Compiler', 'vs'): [re.compile('^(?!\\s|\\d|>)(?P<file>.+?)\\((?P<line>.+)\\):( fatal)? error (?P<type>(C|MSB)\\d+): (?P<details>.+)'), re.compile('MT: command ".+" failed .+\\s+mt :.* error (?P<type>\\w+): .+')], ('Compiler', 'Warnings'): [re.compile('Compiler warnings filter evaluation complete. Num lines evaluated is .+: Errors [1-9].+')], ('Compiler', 'Tasking ctc'): [re.compile('^ERROR: .*: Compiling .* failed: \\(Exit \\d\\): cctc_wrapper\\.bat failed: error executing command$'), re.compile('^ctc E\\d+: \\[\\"(?P<file>.+?)\\" (?P<line>\\d+)/\\d+\\] .+$')], ('Compiler', 'Typescript'): [re.compile('^ERROR:.*: Compiling TypeScript project.*[tsc -p .*] failed: .*')], ('Converter', 'franca_arxml_converter'): [re.compile('ERROR:.*franca_arxml_converter failed: error executing command')], ('Driving Approval Automation Tool (DAAT)', 'Incorrect Config Template. Follow the steps in tool README correctly'): [re.compile('E.*:.*Verification.*failed')], ('Driving Approval Automation Tool (DAAT)', 'Tool failure'): [re.compile('ERROR:root:DAAT.*')], ('Linker', 'gcc'): [re.compile('error: ld returned \\d+ exit status\\n?')], ('Linker', 'ghs'): [re.compile('^.+\\n\\[elxr\\] \\(error #.+\\) (?P<type>.+?):.*\\n(?P<details>([^\\[]*\\n)+\\[elxr\\] \\(error\\).+)?'), re.compile('(\\[elxr\\] \\(error #.+\\).+\\n)+')], ('Linker', 'vs'): [re.compile('^(?P<file>.+?):( fatal)? error (?P<type>LNK\\d+): (?P<details>.+)')], ('Linker', ''): [re.compile('^ERROR: .*: Linking .* failed: \\(Exit \\d\\): .* failed: error executing command')], ('Mentor', 'mentor_generator timeout'): [re.compile('ERROR in mentor_generator\\.py\\s+\\-\\stimeout\\sexpired')], ('Memory', 'double free or corruption'): [re.compile('^double free or corruption \\(out\\)$')], ('Memory', 'Out of memory'): [re.compile('(?:runtime\\/cgo:)? pthread_create failed: Resource temporarily unavailable'), re.compile('^ERROR: .* failed: (\\(Killed\\)|\\(Exit (33|137)\\)): .* failed: error executing command.*$'), re.compile('^cc1plus: out of memory allocating [0-9]+ bytes after a total of [0-9]+ bytes$'), re.compile('^virtual memory exhausted: Cannot allocate memory$'), re.compile('There is insufficient memory for the [a-zA-Z\\s]+ to continue.$'), re.compile('OutOfMemoryError thrown during Starlark evaluation')], ('Misra', 'Errors in analysis of variants'): [re.compile("Errors occurred in [0-9] variants: \\['[a-zA-Z0-9_-]+'\\]")], ('ncar', 'dependency check'): [re.compile('ERROR\\s+Bazel query failed!'), re.compile("ERROR\\s+Dependency rule '\\w+' has violations!")], ('ncar', 'architecture check'): [re.compile('The bazel-check-architecture has found some not allowed packages.\\s+Read.*on how to proceed')], ('ncar', 'lslint'): [re.compile('bazel\\-bin\\/tools\\/ls_lint\\/lslint((.|\\n)*)failed for rules\\: regex')], ('TRLC', ''): [re.compile('^(?P<file>.+?):(?P<line>\\d+).*: trlc (?P<type>(lex |check )?(warning|error|ICE)): (?P<details>.*)')], ('PDX-Generation', 'Timeout E-Sys'): [re.compile('ERROR\\s.*PDX-generation unsuccessful:.*Timeout occurred while waiting for E-Sys.')], ('python', 'Authentication error'): [re.compile('^.*Traceback.+\\n( .+\\n)+.*pip/download.py.+handle_401.*$')], ('python', 'guestfs_launch failed'): [re.compile('RuntimeError: guestfs_launch failed')], ('python', 'HTTPError'): [re.compile('^requests.exceptions.HTTPError: .+$')], ('python', 'pip'): [re.compile('ERROR: Pip failed:.*')], ('python', 'pip install failed'): [re.compile('^ERROR: No matching distribution found for.*$'), re.compile('^Could not install packages due to an.*$')], ('python', 'Failed to install python dependencies'): [re.compile('^ERROR: could not install deps .+$'), re.compile("Preparing metadata \\(setup\\.py\\): finished with status 'error'")], ('python', 'libguestfs'): [re.compile('RuntimeError: guestfs_launch failed.+$')], ('python', 'pytest'): [re.compile('(?s)=========================== short test summary info ============================.*?\\sin\\s.*?=+', re.DOTALL)], ('python', 'thread'): [re.compile("^.*python.+\\n( .+\\n).*error: can't start new thread")], ('python', ''): [re.compile('^(.*Traceback.+\\n( .+\\n)+(?P<type>\\S.+Error.*?):.+\\n?(?P<details>(\\s.+\\n)*)?)+'), re.compile('ERROR: .*: This target is being built for Python 3 but \\(transitively\\) includes Python 2-only sources.'), re.compile('\\/usr\\/bin\\/python.*: No module named .*$')], ('python', 'unrecognized arguments'): [re.compile('(?P<file>[\\w-]+\\.py): error: unrecognized arguments:.+$')], ('python', 'Exception raised'): [re.compile('Exception:(.*)Check missing paths here(.*)')], ('Java', 'EValidator'): [re.compile('^.*ERROR\\s.*\\s-\\sError\\sexecuting\\sEValidator')], ('Java', 'SIGBUS'): [re.compile('^#  SIGBUS\\s\\(0x7\\)\\sat\\spc=0[xX][0-9a-fA-F]+,\\spid=\\d+,\\stid=0[xX][0-9a-fA-F]+$')], ('Java', ''): [re.compile('Error: (?P<type>.+)\\n(?P<details>Exception .+\\n([\\t\\w].+\\n)+)')], ('Symphony', ''): [re.compile('ERROR(\\s*)Logger(\\s*)-?(\\s*)###(?P<details>.+)'), re.compile('^(.+ERROR\\s+generate_symphony_artifacts:.+\\n?)+')], ('System Error', ''): [re.compile('^.+ is not (known|recognized) as [\\s\\S]+?\\.'), re.compile('The system cannot find the (path|file) specified\\.'), re.compile('^.+: (?P<file>(?!.*mount error\\(*).+?): Permission denied'), re.compile('^PermissionError: \\[WinError 32\\] The process cannot access the file because it is being used by another')], ('Shell execution', ''): [re.compile('^(.+\\n)?/bin/sh: \\d+: (?P<type>.+error): (?P<details>.+)'), re.compile('^/bin/bash: line \\d+: (?P<file>.+): (?P<details>.+)'), re.compile('\\/bin\\/bash: .* command not found.*$'), re.compile('The command .* returned a non-zero code: [0-9]*'), re.compile('\\/bin\\/bash: .* No such file or directory.*$'), re.compile("^cp: cannot stat '(?P<file>.+)': .+$")], ('Ansible', 'assert'): [re.compile('^Assertion failed$')], ('Ansible', 'syntax'): [re.compile('The error was: (?P<details>.+)[\\w\\s]+.(?P<file>[\\w/\\.-]+).+: line (?P<line>\\d+)'), re.compile("\\'\\w+\\' is undefined")], ('Ansible', 'templating'): [re.compile('recursive loop detected in template string: .+'), re.compile('templat(e|ing.*) error .+')], ('Ansible', 'creating file/dir'): [re.compile('^There was an issue creating.+$')], ('Ansible', 'missing file'): [re.compile("Could not find or access '(?P<file>.*)' on the Ansible Controller.\\n.*$")], ('Ansible', 'undefined variable'): [re.compile('AnsibleUndefinedVariable:.+$')], ('Git LFS', ''): [re.compile('^(?P<details>.+runtime error: (.*\\n)+?Stopping at .(?P<file>[\\w/-]+).+)'), re.compile('^(?P<details>((?!Git LFS:).+\\n)?error: .+\\nStopping at .(?P<file>[\\w/-]+).+)'), re.compile('^.*\\nerror: failed to fetch some objects.+$'), re.compile('ERROR: Git LFS .*')], ('Test(s) failed', ''): [re.compile('\\/\\/.*\\s+FAILED in [0-9.]+s'), re.compile('\\/\\/.*\\s+FAILED in [0-9]* out of [0-9]*'), re.compile('^\\/\\/.+ FAILED in [0-9\\.]+s$')], ('Test Guide', 'connector'): [re.compile('\\[INFO \\] .* Test Guide connector terminated with error...')], ('Test Guide', 'Tests failing'): [re.compile('Tests still failing after all excludes')], ('Test(s) timeout', ''): [re.compile('\\/\\/.*\\s+TIMEOUT in [0-9]*')], ('Test(s) failed to build', ''): [re.compile('\\/\\/.*\\s+FAILED TO BUILD[$]?')], ('Terminate without exception', ''): [re.compile('terminate called without an active exception')], ('Unknown test failure cause', ''): [re.compile('ScenarioInstance: Shutting down\\.\\.\\.')], ('Dependency Error', 'File not found'): [re.compile('ERROR:.*no such package.*file not found in any of the following directories')], ('Bazel', 'failed on target analysis'): [re.compile('^ERROR: Analysis of target.+$')], ('Bazel', 'crash'): [re.compile('^Internal error .*java\\.lang\\.OutOfMemoryError')], ('Bazel', 'error loading package'): [re.compile('ERROR:(.*)error loading package(.*)')], ('Bazel', 'error finding package'): [re.compile('ERROR: \\/workspace\\/deployment\\/ecu_instances\\/casp_base\\/tc4_eth\\/cdd\\/BUILD:13:17\\: no such package .+: The repository .+ could not be resolved\\: Repository .+ is not defined and reference)], ('Bazel', 'unknown configuration'): [re.compile('ERROR: Config value .* is not defined in any \\.rc file')], ('Bazel', 'server terminated'): [re.compile('^Server terminated abruptly \\(error code:.+$')], ('Bazel', 'error executing shell command'): [re.compile('ERROR: .* error executing shell command: .* failed \\(Exit [0-9]*\\)')], ('Bazel', 'error executing command'): [re.compile('Executing .* failed: \\(Exit [0-9]*\\)\\: .* failed\\: error executing command'), re.compile('ERROR:.{1,500}error executing command \\(.*\\)')], ('Bazel', 'building test target failed'): [re.compile('FAILED: Build did NOT complete successfully\\n.+FAILED TO BUILD.*$')], ('Bazel', 'loading phase errors'): [re.compile('ERROR: command succeeded, but there were loading phase errors')], ('Bazel', 'no targets found'): [re.compile('^ERROR: Skipping.+no targets found beneath.+$'), re.compile('^ERROR: no targets found beneath.+$')], ('Bazel', 'could not build file'): [re.compile("ERROR: .* Couldn't build file .* failed \\(Exit [0-9]*\\).*$")], ('Bazel', 'missing input file'): [re.compile('ERROR:(.*)input file\\(s\\) do not exist'), re.compile("ERROR:(.*)missing input file '.*'")], ('Bazel', 'output was not created'): [re.compile('ERROR:.*output.*was not created')], ('Bazel', 'not all outputs were created or valid'): [re.compile('ERROR:.*not all outputs were created or valid')], ('Bazel', 'name is not defined'): [re.compile("ERROR: (.*) name '(.*)' is not defined(?: \\(did you mean (.*)\\?\\))?")], ('Bazel', 'Failed to extract'): [re.compile("^FATAL: Failed to extract embedded binaries:.+(?P<file>'.+').+$")], ('Bazel', 'Unrecognized option'): [re.compile('ERROR: Unrecognized option:.+$')], ('Bazel', 'target name'): [re.compile('ERROR:.+no such target .* not declared in package .* defined by .*$')], ('Bazel', 'A2l failed'): [re.compile('(?s)a2lfactory\\.exe failed:(?P<details>.+\\n)FAILED: Build did NOT complete successfully', re.DOTALL)], ('Bazel', 'Missing deps. declaration'): [re.compile('ERROR(.*\\n)this rule is missing dependency declarations for the following files included by(.+\\n){0,10}')], ('Bazel', 'BUILD file not found'): [re.compile('ERROR: command succeeded, but not all targets were analyzed')], ('Bazel', 'action cache initialization'): [re.compile("^ERROR: Error during action cache initialization: Failed action cache referential integrity check: Validation mismatch: validation entry [0-9]+ is too large compared to index size [0-9]+\\. Corrupted)], ('Bazel', 'Artifact error'): [re.compile('^ERROR: Unexpected exception, please file an issue with the Bazel team: Artifact.*is neither header nor source file.')], ('Bazel', 'Symlink - missing input file'): [re.compile("^ERROR: (?P<build>.+) Symlinking (?P<file>.+) failed: missing input file '(?P<detail>.+)'$")], ('Bazel', 'Crashed due to an internal error'): [re.compile('FATAL: bazel crashed due to an internal error. Printing stack trace:')], ('Bazel', 'Sandbox execution error'): [re.compile('ERROR: .* failed: I\\/O exception during sandboxed execution\\:.*$')], ('Bazel', 'Build Event Protocol upload timed out'): [re.compile('^ERROR: The Build Event Protocol upload timed out')], ('SRec / S19 not created', ''): [re.compile('^ERROR: .+BUILD:[0-9]+:[0-9]+: output.*\\.s19.+$')], ('Generating functional code failed', ''): [re.compile('^(?P<file>.+?):(?P<line>\\d+).*: ERROR:.+\\/BUILD:[0-9]+:[0-9]+: .* - Generating funct.* failed \\(Exit 2\\):.*$'), re.compile('^(?P<file>.+?):(?P<line>\\d+).*: ERROR:.+\\/BUILD:[0-9]+:[0-9]+: Franca2ARXML.*failed \\(Exit 5\\).*$'), re.compile('^ERROR:.+Generate\\sCode\\sfrom\\sARXML.+bash failed: error executing command'), re.compile('ERROR: .+ Generating arxml files failed\\: \\(Exit .\\)\\: .+ failed: error executing command'), re.compile('^ERROR:.+Generating SWE:.+payload_options failed: error executing command')], ('Win logging error', 'Failed to open Log-Path'): [re.compile('^Get-ChildItem : Cannot find path .+logs.+$')], ('Win logging error', 'Failed to create logman counter'): [re.compile('Error:.*Data Collector already exists')], ('Toolchain', 'Failed to fetch/setup toolchain (e.g. eyeq_sdk, llvm_toolchain)'): [re.compile('^ERROR: .+\\/BUILD:[0-9]+:[0-9]+: \\/\\/bazel\\/toolchains\\/.+$')], ('Dependency', 'Failed to fetch dependency'): [re.compile('^ERROR: .+\\/BUILD:[0-9]+:[0-9]+: .+ depends on .+ which failed to fetch\\..+$')], ('Symlink', 'Failed to create symlink'): [re.compile('^ERROR: .+\\/BUILD:[0-9]+:[0-9]+: failed to create symbolic link .+$'), re.compile('Create symlink from bazel_default_output_base=\\/root\\/.* to target\\=\\/var\\/cache\\/bazel')], ('Formatting error', ''): [re.compile('^Formatting errors found in .+$')], ('redirected output', 'Small Checks - Bazel queries check'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Bazel queries check)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - BUILD-File Validator'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- BUILD-File Validator)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Buildifier-check'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Buildifier-check)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Check CI jobs branch filters'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check CI jobs branch filters)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Check CI jobs maintainers'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check CI jobs maintainers)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Check dependencies for config adp_gcc9'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check dependencies for config adp_gcc9)(?:.|[\\r\\n])], ('redirected output', 'Small Checks - Check the CI governance job'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check the CI governance job)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Check-no-boardnet-switch-is-used'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check-no-boardnet-switch-is-used)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Check_imports_in_py_rules'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check_imports_in_py_rules)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Check_includes_in_cc_rules'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check_includes_in_cc_rules)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Check_strip_include_prefix_in_cc_rules'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check_strip_include_prefix_in_cc_rules)(?:.|[\\r\\n])], ('redirected output', 'Small Checks - Clang format'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Clang-format)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Filesystem-layout'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Filesystem-layout)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Include-guards'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Include-guards)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Python-compile-check'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Python-compile-check)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Python-format-check'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Python-format-check)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Runfile path check'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Runfile path check)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - Validate teamscale architecture'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Validate teamscale architecture)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - collect-FEP-002-violations'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- collect-FEP-002-violations)(?:.|[\\r\\n])*)')], ('redirected output', 'Small Checks - correct-git-lfs-file-tracking'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- correct-git-lfs-file-tracking)(?:.|[\\r\\n])*)')], ('redirected output', 'Fusa Small Checks - Check-docker-image-tags'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check-docker-image-tags)(?:.|[\\r\\n])*)')], ('redirected output', 'Fusa Small Checks - Check-non-qualified-libraries'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check-non-qualified-libraries)(?:.|[\\r\\n])*)')], ('redirected output', 'Fusa Small Checks - Check-Bazel-rules-in-FuSa-critical-components'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Check-Bazel-rules-in-FuSa-critical-components)(?:.|[)], ('redirected output', 'Fusa Small Checks - Windows-filename'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- Windows-filename)(?:.|[\\r\\n])*)')], ('redirected output', 'Third-party Small Checks - third_party best practices'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*(- third_party best practices)(?:.|[\\r\\n])*)')], ('redirected output', 'default'): [re.compile('^@tu-cc-ci-adp-github-eof(?:(?:\\s+type:(?P<type>\\S*))|(?:\\s+file:(?P<file>\\S*))|(?:\\s+line:(?P<line>\\S*)))*\\s+(?P<details>(?:.|[\\r\\n])*)')], ('Matlab', 'Execution error'): [re.compile('(?s)(?<=ERROR MatlabExecutor - Matlab execution error:)(.*$)', re.DOTALL)], ('Matlab', 'Generation error'): [re.compile('(^.*)Generating functional code(.*)failed(.*?$)')], ('Matlab', 'Task(s) fail(ed)'): [re.compile('Code generation for module .* failed.')], ('Docker', 'build'): [re.compile('^(?P<details>Error building (.+)code(.+)message(.+)), logs:'), re.compile('^E: Unable to locate package.+$'), re.compile('[E|W]:.+\\nE: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing\\?$'), re.compile('^error pulling image configuration:.+$'), re.compile('An image does not exist locally with the tag: (.*$)')], ('Kubernetes', ''): [re.compile('^Failed to patch object.+$')], ('clang tidy', ''): [re.compile('^\\[ FAIL \\] Clang Tidy produced.+$')], ('rsync', ''): [re.compile('^rsync error:.+$')], ('astas', 'test'): [re.compile('^FAILED - gui-core interaction tests$')], ("FuSa Violation Found (Check 'bazel_wrapper_log.txt' on the logs folder)", ''): [re.compile('Failed actions detected in bazel_wrapper_log.txt file')], ("FuSa Violation Found (Check 'mitguard_report.html' on the logs folder)", ''): [re.compile('Mitguard check failure')], ("FuSa Violation Found (Check the log on 'Nightwatch' folder)", ''): [re.compile('Nightwatch verdict: rejected\\.')], ('Network Traffic Check', ''): [re.compile('^Network traffic check has failed$')], ('Mount error', 'Permission denied'): [re.compile('Error mounting .+ Permission denied.+$'), re.compile('^.+: (?P<file>mount error.+?): Permission denied')], ('Mount error', 'Mounted FS stat failure'): [re.compile('Error: statfs \\/mnt.+ no such file or directory')], ('Mount error', 'Failed to copy'): [re.compile('failed to copy: .+\\/mnt\\/.+')], ('Mount error', 'Failed to Umount'): [re.compile('Error\\srmdir\\s[a-zA-z-:/]+\\s\\[Errno\\s[0-9]+\\]\\s[a-zA-z\\s\\W]+')], ('Param Server error', ''): [re.compile('ERROR:.+ param_server_gen failed.+$')], ('License', 'Invalid License'): [re.compile('^.*No valid license found for .*')], ('License', 'License not available'): [re.compile('All licenses are currently in use'), re.compile('ERROR: License server valid, no license available'), re.compile("ERROR: Couldn't obtain the license after \\d+ retries.+$")], ('Invoking Requirements Tracing', ''): [re.compile('ERROR.*Invoking Requirements.*driver_tracing failed: \\(Exit 1\\)')], ('Invalid Main repository mapping', ''): [re.compile('^ERROR:.Error computing the main repository mapping:.at.+ is invalid because.+is not a package.+perhaps you meant to put the colon here: .+?$')], ('Cptc Error', ''): [re.compile('cptc .+ cannot open source file.+$')], ('Cmake Error', ''): [re.compile('^make.*Error')], ('windows', 'Restart-Service'): [re.compile("Service 'sshd \\(sshd\\)' cannot be stopped due to the following error: Cannot stop sshd service on ")], ('tar', 'error executing command'): [re.compile('^ERROR: .+ Action .+\\.tar\\.gz failed: \\(Exit 1\\): bash failed: error executing command$')], ('Linux', ''): [re.compile('\\[Errno 2\\] No such file or directory')]}

In [ ]:
# NOTE Regex structure for understanding + imports

import json
import error_patterns
import re

INFRA_PATTERNS_TEST = {
    "Type1":{"r0"},
    "Type2":{"":["r1", "r2","r3"]},
    "Type3":{"Subtype1":["r4"]}
}

BUILD_PATTERNS_TEST = {
    "Bazel": {"error executing command": [
            r"Executing .* failed: \(Exit [0-9]*\)\: .* failed\: error executing command",
            r"ERROR:.{1,500}error executing command \(.*\)"
        ]}
}

print(INFRA_PATTERNS_TEST)

if "r2" == INFRA_PATTERNS_TEST["Type2"][""][1]:
    print("true")
else:
    print("false")


In [ ]:
## outdated version without restructuring

import json
import extract_build_failures.error_patterns as error_patterns
import re
import os
import textwrap


# compile patterns from error_patterns.py
def compile_patterns(patterns_dict):
    compiled_patterns = {}
    for main_category, patterns in patterns_dict.items():
        if isinstance(patterns, dict):
            for sub_category, sub_patterns in patterns.items():
                for pattern in sub_patterns:
                    compiled_patterns.setdefault((main_category, sub_category), []).append(re.compile(pattern))
        else:
            for pattern in patterns:
                compiled_patterns.setdefault((main_category, ""), []).append(re.compile(pattern))
    return compiled_patterns


# check log entry against patterns
def check_log_entry(log_entry, compiled_patterns):
    matches = []
    for (main_category, sub_category), regex_list in compiled_patterns.items():
        for regex in regex_list:
            if regex.search(log_entry):
                category = f"{main_category} - {sub_category}" if sub_category else main_category
                matches.append((category, sub_category, regex.pattern))
    return matches

# process files
def process_log_files(directory_path, compiled_patterns):
    summary = {
        'tasks_summary': {},
        'msg_summary': {}
    }
    for filename in filter(lambda f: f.endswith('.json'), os.listdir(directory_path)):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            log_entries = json.load(file)
            for log in log_entries:
                task_id = log.get('id', 'No ID provided')
                name = log.get('name', 'No task name provided')
                msg = log['msg']
                task_key = f"{name} (ID: {task_id})"
                summary['tasks_summary'].setdefault(task_key, 0)
                summary['tasks_summary'][task_key] += 1
                summary['msg_summary'].setdefault(msg, 0)
                summary['msg_summary'][msg] += 1

                stdout_text = "\n".join(log.get('stdout_lines', []))
                matches = check_log_entry(stdout_text, compiled_patterns)
                for match in matches:
                    main_category, sub_category, pattern = match
                    print(f"Current File: {file_path}")
                    print(task_key)
                    print(f"Main Category: {main_category}, Subcategory: {sub_category}, Pattern: {pattern}")
    return summary

# format summary for readability
def format_summary_to_screen_width(summary, terminal_width=150):
    formatted_summary = ""
    for key, value in summary.items():
        if isinstance(value, dict):
            formatted_summary += f"{key}:\n"
            for sub_key, sub_value in value.items():
                wrapped_sub_value = textwrap.fill(str(sub_value), terminal_width - 4)
                formatted_summary += f"  {sub_key}: {wrapped_sub_value}\n"
        else:
            wrapped_value = textwrap.fill(str(value), terminal_width)
            formatted_summary += f"{key}: {wrapped_value}\n"
        formatted_summary += "-" * terminal_width + "\n"
    return formatted_summary

# Usage:
compiled_infra_patterns = compile_patterns(INFRA_PATTERNS)
compiled_build_patterns = compile_patterns(BUILD_PATTERNS)
directory_path = 'preprocessed_logs'
all_compiled_patterns = {**compiled_infra_patterns, **compiled_build_patterns}
logs_directory_path = 'logs'
output_directory_path = 'preprocessed_logs'

summary = process_log_files(directory_path, all_compiled_patterns)
formatted_summary = format_summary_to_screen_width(summary)
print(formatted_summary)

In [ ]:
# check a specific file (tbh: outdated)
import json

def extract_error_info_from_file(json_file_path):
    """
    Reads a JSON file and extracts error information from the "plays" section, including all tasks where "failed" is true,
    and retrieves "stderr", "stdout_lines", "msg", and the task "name" for the host's node.

    :param json_file_path: The file path to the JSON data file.
    :return: A list of error information dictionaries.
    """
    error_info_list = []

    try:
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        if not isinstance(data, list):
            raise ValueError("JSON data must be a list.")

        for item in data:
            for play in item.get('plays', []):
                for task in play.get('tasks', []):
                    for node, host_info in task.get('hosts', {}).items():
                        if host_info.get('failed'):
                            error_info = {
                                'node': node,
                                'stderr': host_info.get('stderr', ''),
                                'stdout_lines': host_info.get('stdout_lines', []),
                                'msg': host_info.get('msg', 'No message provided'),
                                'id': task.get('role', {}).get('id', 'No id provided'),
                                'name': task.get('role', {}).get('name', 'No task name provided')
                            }
                            error_info_list.append(error_info)
    except Exception as e:
        print(f"An error occurred while processing {json_file_path}: {e}")

    return error_info_list

# Example usage with a specific JSON file path
specific_log_file_path = 'logs/882dafcc748b4f0695ce486241a05ad2.json'

# Extract error info from the specific JSON file
error_info = extract_error_info_from_file(specific_log_file_path)
print(json.dumps(error_info, indent=4))